In [1]:
import pandas as pd

df = pd.read_csv('SWaT/swat2.csv')
print(df.head())
df = df.values
print(df.shape)
ano = 0
for u in range(len(df)):
    if(df[u][-1]==1):
        ano = ano+1
print(ano)
print(ano/len(df))

     FIT101    LIT101   MV101  P101  P102    AIT201    AIT202    AIT203  \
0  2.427057  522.8467     2.0   2.0   1.0  262.0161  8.396437  328.6337   
1  2.446274  522.8860     2.0   2.0   1.0  262.0161  8.396437  328.6337   
2  2.489191  522.8467     2.0   2.0   1.0  262.0161  8.394514  328.6337   
3  2.534350  522.9645     2.0   2.0   1.0  262.0161  8.394514  328.6337   
4  2.569260  523.4748     2.0   2.0   1.0  262.0161  8.394514  328.6337   

     FIT201   MV201  ...  P501  P502    PIT501    PIT502    PIT503    FIT601  \
0  2.445391     2.0  ...   2.0   1.0  250.8652  1.649953  189.5988  0.000128   
1  2.445391     2.0  ...   2.0   1.0  250.8652  1.649953  189.6789  0.000128   
2  2.442316     2.0  ...   2.0   1.0  250.8812  1.649953  189.6789  0.000128   
3  2.442316     2.0  ...   2.0   1.0  250.8812  1.649953  189.6148  0.000128   
4  2.443085     2.0  ...   2.0   1.0  250.8812  1.649953  189.5027  0.000128   

   P601  P602  P603  Normal/Attack  
0   1.0   1.0   1.0            

In [2]:
df = pd.read_csv('SWaT/swat_train2.csv')
print(df.head())
df = df.values
print(df.shape)
ano = 0
for u in range(len(df)):
    if(df[u][-1]==1):
        ano = ano+1
print(ano)
print(ano/len(df))

   FIT101    LIT101   MV101  P101  P102    AIT201    AIT202    AIT203  FIT201  \
0     0.0  124.3135     1.0   1.0   1.0  251.9226  8.313446  312.7916     0.0   
1     0.0  124.3920     1.0   1.0   1.0  251.9226  8.313446  312.7916     0.0   
2     0.0  124.4705     1.0   1.0   1.0  251.9226  8.313446  312.7916     0.0   
3     0.0  124.6668     1.0   1.0   1.0  251.9226  8.313446  312.7916     0.0   
4     0.0  124.5098     1.0   1.0   1.0  251.9226  8.313446  312.7916     0.0   

    MV201  ...  P501  P502    PIT501  PIT502  PIT503    FIT601  P601  P602  \
0     1.0  ...   1.0   1.0  9.100231     0.0  3.3485  0.000256   1.0   1.0   
1     1.0  ...   1.0   1.0  9.100231     0.0  3.3485  0.000256   1.0   1.0   
2     1.0  ...   1.0   1.0  9.100231     0.0  3.3485  0.000256   1.0   1.0   
3     1.0  ...   1.0   1.0  9.100231     0.0  3.3485  0.000256   1.0   1.0   
4     1.0  ...   1.0   1.0  9.100231     0.0  3.3485  0.000256   1.0   1.0   

   P603  Normal/Attack  
0   1.0            

In [11]:
df = pd.read_csv('SWaT/swat_raw.csv')
# 表头设置为第一行，然后删除第一行
df.columns = df.iloc[0]
df = df.drop(0)
print(df.head())

C:\Users\14680\AppData\Local\Temp\ipykernel_20052\1848050369.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SWaT/swat_raw.csv')


0                Timestamp    FIT101    LIT101  MV101 P101 P102    AIT201  \
1   28/12/2015 10:00:00 AM  2.427057  522.8467      2    2    1  262.0161   
2   28/12/2015 10:00:01 AM  2.446274   522.886      2    2    1  262.0161   
3   28/12/2015 10:00:02 AM  2.489191  522.8467      2    2    1  262.0161   
4   28/12/2015 10:00:03 AM   2.53435  522.9645      2    2    1  262.0161   
5   28/12/2015 10:00:04 AM   2.56926  523.4748      2    2    1  262.0161   

0    AIT202    AIT203    FIT201  ... P501 P502    PIT501    PIT502    PIT503  \
1  8.396437  328.6337  2.445391  ...    2    1  250.8652  1.649953  189.5988   
2  8.396437  328.6337  2.445391  ...    2    1  250.8652  1.649953  189.6789   
3  8.394514  328.6337  2.442316  ...    2    1  250.8812  1.649953  189.6789   
4  8.394514  328.6337  2.442316  ...    2    1  250.8812  1.649953  189.6148   
5  8.394514  328.6337  2.443085  ...    2    1  250.8812  1.649953  189.5027   

0       FIT601 P601 P602 P603 Normal/Attack  
1  0.00012

In [15]:
# 打印所有的列名
print(df.columns)
# 获取第一行的数据
print(df.iloc[0])
# 获取第一行第一个数据
print(df.iloc[0][0])

Index([' Timestamp', 'FIT101', 'LIT101', ' MV101', 'P101', 'P102', ' AIT201',
       'AIT202', 'AIT203', 'FIT201', ' MV201', ' P201', ' P202', 'P203',
       ' P204', 'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301',
       'MV302', ' MV303', 'MV304', 'P301', 'P302', 'AIT401', 'AIT402',
       'FIT401', 'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401', 'AIT501',
       'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504',
       'P501', 'P502', 'PIT501', 'PIT502', 'PIT503', 'FIT601', 'P601', 'P602',
       'P603', 'Normal/Attack'],
      dtype='object', name=0)
0
 Timestamp        28/12/2015 10:00:00 AM
FIT101                          2.427057
LIT101                          522.8467
 MV101                                 2
P101                                   2
P102                                   1
 AIT201                         262.0161
AIT202                          8.396437
AIT203                          328.6337
FIT201                          2.4453

C:\Users\14680\AppData\Local\Temp\ipykernel_20052\2484512602.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[0][0])
